# Simulation of a quadruped Robot


Using the simulator the goal of this challenge is to come up with an Agent for controlling a Quadrupedal Robot on a flat environement. The best solution will be the solution that goes as far as possible given a number of Timesteps

## Using the Environment

You are provided with an environment that uses the [OpenAI Gym Interface](https://gym.openai.com/). 

Once initialized, you can interact with the environment using the step function that takes a set of actions ( a numpy array of size 12 between -1.0 and 1.0 that corresponds to target angles for each joints. ) 
The order of the joint angles is as follow [knee, hip, shoulder] for each of the legs in order [Front Left, Rear Right, Front Right, Rear Left]


In [1]:
import aida_env.aida_gym_env as e
import pybullet as p
import numpy as np
import gym

# Writing an Actor-Critic Agent

You can write an Agent to control the actions sent to the environment. 

In [2]:
import numpy as np 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as K

import tensorflow as tf

import random
from collections import deque

/Library/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# determines how to assign values to each state, i.e. takes the state
# and action (two-input model) and determines the corresponding value
class ActorCritic:
    def __init__(self, env, sess):
        self.env  = env
        self.sess = sess

        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = .995
        self.gamma = .95
        self.tau   = .125

        # ===================================================================== #
        #                               Actor Model                             #
        # Chain rule: find the gradient of chaging the actor network params in  #
        # getting closest to the final value network predictions, i.e. de/dA    #
        # Calculate de/dA as = de/dC * dC/dA, where e is error, C critic, A act #
        # ===================================================================== #

        self.memory = deque(maxlen=2000)
        self.actor_state_input, self.actor_model = self.create_actor_model()
        _, self.target_actor_model = self.create_actor_model()

        self.actor_critic_grad = tf.placeholder(tf.float32, 
            [None, self.env.action_space.shape[0]]) # where we will feed de/dC (from critic)
        
        actor_model_weights = self.actor_model.trainable_weights
        self.actor_grads = tf.gradients(self.actor_model.output, 
            actor_model_weights, -self.actor_critic_grad) # dC/dA (from actor)
        grads = zip(self.actor_grads, actor_model_weights)
        self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)

        # ===================================================================== #
        #                              Critic Model                             #
        # ===================================================================== #

        self.critic_state_input, self.critic_action_input, \
            self.critic_model = self.create_critic_model()
        _, _, self.target_critic_model = self.create_critic_model()

        self.critic_grads = tf.gradients(self.critic_model.output, 
            self.critic_action_input) # where we calcaulte de/dC for feeding above
        
        # Initialize for later gradient calculations
        self.sess.run(tf.initialize_all_variables())

    # ========================================================================= #
    #                              Model Definitions                            #
    # ========================================================================= #

    def create_actor_model(self):
        state_input = Input(shape=self.env.observation_space.shape)
        h1 = Dense(64, activation='relu')(state_input)
        h2 = Dense(128, activation='relu')(h1)
        h3 = Dense(64, activation='relu')(h2)
        output = Dense(self.env.action_space.shape[0], activation='sigmoid')(h3)
        
        model = Model(input=state_input, output=output)
        adam  = Adam(lr=0.001)
        model.compile(loss="mse", optimizer=adam)
        return state_input, model

    def create_critic_model(self):
        state_input = Input(shape=self.env.observation_space.shape)
        state_h1 = Dense(64, activation='relu')(state_input)
        state_h2 = Dense(128)(state_h1)
        
        action_input = Input(shape=self.env.action_space.shape)
        action_h1    = Dense(128)(action_input)
        
        merged    = Add()([state_h2, action_h1])
        merged_h1 = Dense(48, activation='relu')(merged)
        output = Dense(1, activation='relu')(merged_h1)
        model  = Model(input=[state_input,action_input], output=output)
        
        adam  = Adam(lr=0.001)
        model.compile(loss="mse", optimizer=adam)
        return state_input, action_input, model

    # ========================================================================= #
    #                               Model Training                              #
    # ========================================================================= #

    def remember(self, cur_state, action, reward, new_state, done):
        self.memory.append([cur_state, action, reward, new_state, done])

    def _train_actor(self, samples):
        for sample in samples:
            cur_state, action, reward, new_state, _ = sample
            predicted_action = self.actor_model.predict(cur_state)
            grads = self.sess.run(self.critic_grads, feed_dict={
                self.critic_state_input:  cur_state,
                self.critic_action_input: predicted_action
            })[0]

            self.sess.run(self.optimize, feed_dict={
                self.actor_state_input: cur_state,
                self.actor_critic_grad: grads
            })
            
    def _train_critic(self, samples):
        for sample in samples:
            cur_state, action, reward, new_state, done = sample
            if not done:
                target_action = self.target_actor_model.predict(new_state)
                future_reward = self.target_critic_model.predict(
                    [new_state, target_action])[0][0]
                reward += self.gamma * future_reward
            self.critic_model.fit([cur_state, action], [reward], verbose=0)
        
    def train(self):
        batch_size = 32
        if len(self.memory) < batch_size:
            return

        rewards = []
        samples = random.sample(self.memory, batch_size)
        self._train_critic(samples)
        self._train_actor(samples)

    # ========================================================================= #
    #                         Target Model Updating                             #
    # ========================================================================= #

    def _update_actor_target(self):
        actor_model_weights  = self.actor_model.get_weights()
        actor_target_weights = self.target_critic_model.get_weights()
        
        for i in range(len(actor_target_weights)):
            actor_target_weights[i] = actor_model_weights[i]
        self.target_critic_model.set_weights(actor_target_weights)

    def _update_critic_target(self):
        critic_model_weights  = self.critic_model.get_weights()
        critic_target_weights = self.critic_target_model.get_weights()
        
        for i in range(len(critic_target_weights)):
            critic_target_weights[i] = critic_model_weights[i]
        self.critic_target_model.set_weights(critic_target_weights)

    def update_target(self):
        self._update_actor_target()
        self._update_critic_target()

    # ========================================================================= #
    #                              Model Predictions                            #
    # ========================================================================= #

    def act(self, cur_state):
        self.epsilon *= self.epsilon_decay
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return self.actor_model.predict(cur_state).reshape(self.env.action_space.shape)


### We create an instance of agent ActorCritic

In [4]:
sess = tf.Session()
K.set_session(sess)
#env = gym.make("Pendulum-v0")
env = e.AidaBulletEnv(render=False, on_rack=False)
actor_critic = ActorCritic(env, sess)

num_trials = 10000
trial_len  = 500
    
cur_state = env.step(np.zeros(12))[0]
action = env.action_space.sample()

    

urdf_root=/Library/anaconda/lib/python2.7/site-packages/pybullet_data
aida_env/urdf/plane.urdf
/Library/anaconda/lib/python2.7/site-packages/pybullet_data/plane.urdf
(0.01, 1.0)
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


/Library/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/Library/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:74: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


Instructions for updating:
Use `tf.global_variables_initializer` instead.


### We want to train our agent

In [5]:
    it = 0
    rewards=[]
    while it < 2000:
        env.render()
        #cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
        #print(cur_state)
        action = actor_critic.act(cur_state)
        #action = action.reshape((1, env.action_space.shape[0]))
        
        new_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        #new_state = new_state.reshape((1, env.observation_space.shape[0]))
        action = action.reshape((1, env.action_space.shape[0]))
        new_state = new_state.reshape((1, env.observation_space.shape[0]))
        cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
        actor_critic.remember(cur_state, action, -reward, new_state, done)
        actor_critic.train()
        if it%50==0:
            print('iteration: '+ str(it)+ ', average reward: '+ str(np.mean(rewards)))
        it+=1


iteration: 0, average reward: 0.133501879842381
iteration: 50, average reward: 0.8523709762706646
iteration: 100, average reward: 0.6819874542343691
iteration: 150, average reward: 0.37727862707173676
iteration: 200, average reward: 0.262589822351596


KeyboardInterrupt: 

### Let's watch how it works

In [6]:
env = e.AidaBulletEnv(render=True, on_rack=False)

urdf_root=/Library/anaconda/lib/python2.7/site-packages/pybullet_data
aida_env/urdf/plane.urdf
/Library/anaconda/lib/python2.7/site-packages/pybullet_data/plane.urdf
(0.01, 1.0)
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [7]:
cur_state = env.step(np.zeros(12))[0]
for i in range(500):
    cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
    action = actor_critic.act(cur_state)
    new_state, reward, done, _ = env.step(action)
    cur_state=new_state
    #print(reward)

KeyboardInterrupt: 

In [ ]:
p.disconnect()

# Agents

An agent must contain a function named get_actions() (without any args, see example above) that will return the next set of actions to be performed on the environment. Make sure that this is respected so that the testing function work ( see below).
The test function will test how far the robot goes and return the distance from the start. This will be used to evaluate your solution.

In [ ]:
from aida_env.utils import test_agent_on_env

In [ ]:
test_agent_on_env(agent)